In [1]:
!pip3 install -r requirements.txt

In [2]:
!pip3 install google-generativeai

  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.3 which is incompatible.
streamlit 1.22.0 requires protobuf<4,>=3.12, but you have protobuf 4.25.3 which is incompatible.


In [3]:
import os
from dotenv import load_dotenv
load_dotenv() 

True

In [4]:
from langchain.llms import GooglePalm

llm = GooglePalm(google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0.2)

In [5]:
conversation = llm("Create a conversation between an old man and a toddler")

In [6]:
##Connect with database and ask some basic questions
conversation

"**Old Man:** Hello there, little one.\n\n**Toddler:** Hi!\n\n**Old Man:** What's your name?\n\n**Toddler:** My name is [Toddler's name].\n\n**Old Man:** It's nice to meet you, [Toddler's name]. What are you doing today?\n\n**Toddler:** I'm playing with my toys.\n\n**Old Man:** That sounds like fun. I used to love playing with toys when I was your age.\n\n**Toddler:** What kind of toys did you like to play with?\n\n**Old Man:** I liked to play with blocks, cars, and toy soldiers.\n\n**Toddler:** Those sound like fun toys. I like to play with blocks too.\n\n**Old Man:** Blocks are great. You can build all sorts of things with them.\n\n**Toddler:** I like to build towers.\n\n**Old Man:** Towers are fun. I used to build towers all the time when I was your age.\n\n**Toddler:** I'm going to build a big tower.\n\n**Old Man:** I'm sure you will. You're a very good builder.\n\n**Toddler:** Thanks!\n\n**Old Man:** You're welcome.\n\n**Toddler:** (builds a tower)\n\n**Old Man:** Wow! That's a gr

In [7]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [8]:
db_user = os.getenv("db_user")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_name = os.getenv("db_name")

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE autoinsurance (
	`Customer` VARCHAR(7) NOT NULL, 
	`State` VARCHAR(20), 
	`Customer_Lifetime_Value` DOUBLE, 
	`Response` VARCHAR(3), 
	`Coverage` VARCHAR(10), 
	`Education` VARCHAR(30), 
	`Effective_To_Date` DATE, 
	`EmploymentStatus` VARCHAR(15), 
	`Gender` VARCHAR(6), 
	`Income` MEDIUMTEXT, 
	`Location_Code` VARCHAR(15), 
	`Marital_Status` VARCHAR(15), 
	`Monthly_Premium_Auto` INTEGER, 
	`Months_Since_Last_Claim` INTEGER, 
	`Months_Since_Policy_Inception` INTEGER, 
	`Number_of_Open_Complaints` INTEGER, 
	`Number_of_Policies` INTEGER, 
	`Policy_Type` VARCHAR(30), 
	`Policy` VARCHAR(30), 
	`Renew_Offer_Type` VARCHAR(10), 
	`Sales_Channel` VARCHAR(15), 
	`Total_Claim_Amount` DOUBLE, 
	`Vehicle_Class` VARCHAR(25), 
	`Vehicle_Size` VARCHAR(10), 
	PRIMARY KEY (`Customer`)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from autoinsurance table:
Customer	State	Customer_Lifetime_Value	Response	Coverage	Education	Effective_To_Date	EmploymentStatus	Ge

In [9]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble."""

In [10]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("how many states data is available and name them?")



> Entering new SQLDatabaseChain chain...
how many states data is available and name them?
SQLQuery:SELECT DISTINCT State FROM autoinsurance
SQLResult: [('California',), ('Nevada',), ('Washington',), ('Arizona',), ('Oregon',)]
Answer:5, California, Nevada, Washington, Arizona, Oregon
> Finished chain.


In [11]:
qns2 = db_chain.run("can u give the count of different sales channel from the data")



> Entering new SQLDatabaseChain chain...
can u give the count of different sales channel from the data
SQLQuery:SELECT COUNT(DISTINCT Sales_Channel) FROM autoinsurance
SQLResult: [(4,)]
Answer:4
> Finished chain.


In [12]:
qns3 = db_chain("whow many different sales channel are present")



> Entering new SQLDatabaseChain chain...
whow many different sales channel are present
SQLQuery:SELECT COUNT(DISTINCT Sales_Channel) FROM autoinsurance
SQLResult: [(4,)]
Answer:4
> Finished chain.


In [13]:
qns4 = db_chain("how many male and female are there?")



> Entering new SQLDatabaseChain chain...
how many male and female are there?
SQLQuery:SELECT COUNT(DISTINCT Gender) FROM autoinsurance
SQLResult: [(2,)]
Answer:2
> Finished chain.


In [14]:
qns5 = db_chain("what is the average monthly premium")



> Entering new SQLDatabaseChain chain...
what is the average monthly premium
SQLQuery:SELECT AVG(Monthly_Premium_Auto) FROM autoinsurance
SQLResult: [(Decimal('93.2193'),)]
Answer:93.2193
> Finished chain.


In [16]:
qns5 = db_chain("what is the average monthly premium in the state Washington ")



> Entering new SQLDatabaseChain chain...
what is the average monthly premium in the state Washington 
SQLQuery:SELECT AVG(Monthly_Premium_Auto) FROM autoinsurance WHERE State = 'Washington'
SQLResult: [(Decimal('92.0313'),)]
Answer:92.0313
> Finished chain.


In [17]:
qns6 = db_chain("how many years data is present ")



> Entering new SQLDatabaseChain chain...
how many years data is present 
SQLQuery:SELECT DISTINCT YEAR(Effective_To_Date) FROM autoinsurance
SQLResult: [(2011,)]
Answer:1
> Finished chain.
